In [1]:
from itertools import combinations
import numpy as np

In [2]:
# Convert to chain complex
XF={(0,):0,(1,):0,(2,):0,(3,):0,(4,):0,(5,):0,(0,1):1,(0,2):1.1,(1,2):1.2,(1, 3):1.3,(4, 5):1.4,
   (1, 4):1.5,(2, 4):1.6,(3, 4):1.8,(2, 5):1.7,(0,1,2):2,(1, 3,4):2.1,(2, 4,5):2.2}

In [3]:
def field_inversion(coeff_field):
    def inverse(a, n):
        for x in range(n):
            if (a * x) % n == 1:
                return x
    inversion_table = [
        inverse(a + 1, coeff_field) for a in range(coeff_field - 1)]
    def inversion_fct(a):
        return inversion_table[a-1]
    return inversion_fct

Conversion of Filtered Simplicial Complex to Totally Filtered Chain Complex

In [4]:
def chain_complex_conversion(XF,k):#order simplices by filtration, construct vector space
    def filt_fct(sigma):
        return XF[sigma]
    ordered_simplices=sorted(list(XF.keys()),key=filt_fct)
    
    simplices=dict((sigma,i) for i,sigma in enumerate(ordered_simplices))
    indices=dict((i,sigma) for i,sigma in enumerate(ordered_simplices))
    vspace=dict((sigma,(sigma,1)) for sigma in ordered_simplices)
    
    def deg(n):
        sigma=indices[n]
        return len(sigma)-1

    def bound(n):
        sigma=indices[n]
        boundary=list(combinations(sigma,len(sigma)-1))
        vboundary=[simplices[face] for face in boundary]
        return list(vboundary)
    
    totally_filtered_chain_complex=dict((('free Vspace',vspace),
                                        ('functions',(deg,bound)),
                                        ('coeff field',k),
                                        ('inversion field',field_inversion(k)),
                                        ('indices',indices),
                                        ('simplices',simplices)))
    
    return totally_filtered_chain_complex

Extend Homology Basis

In [5]:
def homology_basis_extension(ttfc,P,f,i):
    P.append(i)
    f[i]=frozenset(((i, 1),))
    return (P,f)

Determine the Image of the Boundary of Given Simplex under Prior Basis

In [6]:
def boundary_under_basis(bound,keys,f,coeff_field,i):
    v=dict()
    
    boundary=bound(i)
    boundary_under_f=set.intersection(set(boundary),set(keys))
    for face in boundary_under_f:
        for h_face,h_value in f[face]:
            val=(v.pop(h_face,0)+(-1)**boundary.index(face)*h_value)%coeff_field
                
            if val>0:
                v[h_face]=val
    if not v:
        return False
    else:
        return v

Contract Homology Basis

In [7]:
def homology_basis_contraction(ttfc,bub,P,f,R,keys,indices,i):
    support=set(bub)
    lowv=max(support)
    P.remove(lowv)
    #f=f for dimension<d is trivially fulfilled
    #print('before pi: '+str(f))
    f,R,keys,indices=pi_transformation(ttfc,bub,support,f,R,keys,indices,lowv)
    #print('after pi: '+str(f))
    return (P,f,R,keys,indices,lowv)

Elimination of lowv from basis k{P_{i-1}}

In [8]:
def pi_transformation(ttfc,bub,support,f,R,keys,indices,lowv):
    inverselowv=ttfc['inversion field'](bub[lowv])
    support.remove(lowv),bub.pop(lowv)
            
    v={tau:(val*inverselowv)%ttfc['coeff field'] for tau,val in bub.items()}# v/v_{l}
    things=R.pop(lowv)
    #print('R.pop(lowv): '+str(things))
    for col,val in things: #Identify which basis function will be affected by pi transformation
        #print('col,val: '+str(col)+', '+str(val))
        col_wo=dict(f[col])
        col_wo.pop(lowv) #the transformation is designed to kill lowv, so no computation necessary
        
        if not col_wo: #the case when w_{l} is zero so we know exactly what happens under pi transform
            reduced={tau:(-val*value)%ttfc['coeff field'] for tau,value in v.items()} #0-(w_{l}/v_{l})v term
        else:
            v_mult={tau:(-val*value)%ttfc['coeff field'] for tau,value in v.items()}
            red= col_wo.copy()
            red.update(v_mult)
            for tau in support.intersection(col_wo.keys()):          
                redtau=(col_wo.get(tau,0)+v_mult.get(tau,0))%ttfc['coeff field'] #w-(w_{l}/v_{l})v term
                if redtau==0:
                    red.pop(tau)
                else:
                    red[tau]=redtau
            reduced=red
        #print('reduced column: '+str(reduced))
        f[col]=frozenset(reduced.items()) #Apply transformation to lowv basis term
        #print('supportUcol_wo: '+str(support.intersection(col_wo)))
        for row in support.intersection(col_wo):
            R[row].remove(col,col_wo[row])
        if not f[col]: #only non-zero basis elements are remembered, so f extended to \{i\}\mapsto 0 is then implied
            f.pop(col) 
            keys.remove(col)
            indices.pop(col)
        else:
            for row in support.intersection(reduced):
                R[row].add((col, reduced[row])) 
    return (f,R,keys,indices)

HomologyBasis Algorithm

In [9]:
def HomologyBasis(XF,k=11):
    ttfc=chain_complex_conversion(XF,k)
    #print(ttfc)
    deg,bound=ttfc['functions']
    P=[0]
    f={0:frozenset(((0, 1),))}
    R={0:set(((0,1),))}
    pairs=[]
    keys=[0]
    
    for i in range(1,len(XF.keys())):
        #print(' ')
        sigma=ttfc['indices'][i]
        index=ttfc['simplices'][sigma]
        #print('simplex: '+str(sigma))
        #print('prior basis: '+str(P))
        
        if len(sigma)==1:
            P,f=homology_basis_extension(ttfc,P,f,i)
            R[i]=set(((i,1),))
            keys.append(i)
            continue
        
        bub=boundary_under_basis(bound,keys,f,ttfc['coeff field'],i)
        #print('f(\partial\sigma) :'+str(bub))
        
        if bub==False:#Extend
            #print('Run extend')
            P,f=homology_basis_extension(ttfc,P,f,i)
            R[i]=set(((i,1),))
            keys.append(i)
            continue
            
        else:#Contract
            #print('Run contract')
            P,f,R,keys,ttfc['indices'],l=homology_basis_contraction(ttfc,bub,P,f,R,keys,ttfc['indices'],i)
            pairs.append((l,i))
        #print('R: '+str(R))
    for j in P:
        pairs.append((j,np.inf))
    def first(e):
        return e[0]
    pairs.sort(key=first)
    return P,f,pairs,ttfc

In [10]:
HB=HomologyBasis(XF)